In [177]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [178]:
allFacilities = pd.read_csv("output/describeNetwork.csv", dtype={'primary_naics': object})
naics = pd.read_csv("data/naics.csv", dtype={'2012 NAICS US Code': object})

In [179]:
# Dioxin reporters from 2013 TRI 
tri2013 = pd.read_csv('data/toxic-release-inventory.ny.2013.geoid.csv')
# dioxins_2013 = tri2013[tri2013['chemical']=='DIOXIN AND DIOXIN-LIKE COMPOUNDS']
# len(dioxins_2013) # Number of dioxin reporting facilities

In [180]:
allChems = pd.unique(tri2013['chemical'].values.ravel())
len(allChems)

181

In [181]:
allComms = sorted(pd.unique(allFacilities['Community'].values.ravel()))
allComms

[0, 1, 2, 3, 4, 5]

In [182]:
# chem = 'DIOXIN AND DIOXIN-LIKE COMPOUNDS'

In [183]:
naics = naics.drop(naics.columns[[0,3,4]], axis=1)
naics.columns = ["naics", "industry"]
allFacilities = allFacilities.merge(naics, left_on=["primary_naics"], right_on="naics", how="left")
# allFacilities = allFacilities.drop("naics", axis=1)

In [184]:
allFacilities = allFacilities.drop(['Unnamed: 0', 'naics', 'Betweeness','Closeness','Eigenvector'], axis=1)

In [185]:
# print 'Num of total facilities:', len(allFacilities)
# print 'Num of dioxin-emitting facilities:', len(withChem)
# print 'Num of non dioxin-emitting facilities:', len(withoutChem)

In [186]:
# allFacilities[allFacilities['Chemicals'].str.contains('DIOXIN AND DIOXIN-LIKE COMPOUNDS')]

In [195]:
cols = ['nonReportChem', 'Community', 'Facility', 'parent_company_name', 'industry']

In [196]:
def chemPickCommunity(comm, chem):
    withChem = allFacilities[allFacilities['Chemicals'].str.contains(chem)]
    withoutChem = allFacilities[~allFacilities['Chemicals'].str.contains(chem)]
    withChemComm = withChem[withChem['Community'] == comm]
    withoutChemComm = withoutChem[withoutChem['Community'] == comm]
    chemNaics = pd.unique(withChemComm['primary_naics'].values.ravel())
    nonReporters = withoutChemComm[withoutChemComm['primary_naics'].isin(chemNaics)]
    nonReporters['nonReportChem'] = chem
    return nonReporters[cols]
#     return withoutChemComm.merge(withChemComm, on='primary_naics', how='inner')

In [197]:
# chemPickCommunity(0).head()

In [211]:
allNonRepList = []

In [212]:
for i in allComms:
    for j in allChems:
#     print len(chemPickCommunity(i))
#         currNonReporters = chemPickCommunity(i,j)
#         allNonReporters.append(currNonReporters)
#         print currNonReporters
        allNonRepList.append(chemPickCommunity(i,j))

/Users/afair/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/afair/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from IPython.kernel.zmq import kernelapp as app
/Users/afair/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()


In [213]:
final = pd.concat(i for i in allNonRepList)

In [214]:
final.to_csv('nonReporters.csv')